# Assignment 2 Deepankar Varma 102003431 

### Step 1: Import the required libraries

In [34]:
import cv2
import dlib
import numpy as np


### Step 2: Load the dataset and preprocess the images

In [35]:
import os

image_folder = 'Images/H'
images = []
for filename in os.listdir(image_folder):
    img = cv2.imread(os.path.join(image_folder, filename))
    if img is not None:
        images.append(img)


### Step 3: Face detection and alignment using Haar Cascades and Dlib

In [36]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

face_rects = []
for image in images:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    for (x,y,w,h) in faces:
        face_rects.append((x,y,w,h))


### Step 4: Feature extraction using Histogram of Oriented Gradients (HOG)

In [37]:
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

aligned_faces = []
for image, (x,y,w,h) in zip(images, face_rects):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    landmarks = predictor(gray, dlib.rectangle(x,y,x+w,y+h))
    landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])
    
    # calculate the mean point of the two corners of the eyes in the image
    left_eye = np.mean(landmarks[36:42], axis=0)
    right_eye = np.mean(landmarks[42:48], axis=0)

    # calculate the angle between the line connecting the eyes and the horizontal axis
    dy = right_eye[1] - left_eye[1]
    dx = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dy, dx)) - 180

    # calculate the scale factor
    desired_left_eye = (0.35, 0.35)
    dist = np.sqrt((dx ** 2) + (dy ** 2))
    desired_dist = (desired_left_eye[1] - desired_left_eye[0]) * image.shape[1]
    scale = desired_dist / dist

    # calculate the rotation matrix
    center = (left_eye + right_eye) // 2
    M = cv2.getRotationMatrix2D(tuple(center), angle, scale)

    # apply the transformation to the image
    tX = image.shape[1] // 2 - center[0]
    tY = image.shape[0] * 0.35 - center[1]
    M[0, 2] += tX
    M[1, 2] += tY

    aligned_face = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_CUBIC)
    aligned_faces.append(aligned_face)


### Step 5: Split the dataset into training and testing sets

In [40]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

features = []
for aligned_face in aligned_faces:
    # resize the image to the input size of the VGG16 model
    input_image = cv2.resize(aligned_face, (224, 224))

    # preprocess the input image
    input_image = preprocess_input(input_image)

    # extract features using the VGG16 model
    feature = model.predict(np.expand_dims(input_image, axis=0))[0]

    features.append(feature)


1/1 [==============================] - 0s 266ms/step


In [32]:
from scipy.spatial.distance import euclidean

def distance(feature1, feature2):
    return euclidean(feature1, feature2)

def match(feature, database, threshold):
    distances = [distance(feature, db_feature) for db_feature in database]
    min_distance = min(distances)
    if min_distance < threshold:
        return np.argmin(distances)
    else:
        return None

database = np.array(features)
threshold = 0.6

# test the model on a new image
test_image = cv2.imread('test_image.jpg')
gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
for (x,y,w,h) in faces:
    face = test_image[y:y+h, x:x+w]
    aligned_face = align_face(face, predictor)
    input_image = cv2.resize(aligned_face, (224, 224))
    input_image = preprocess_input(input_image)
    feature = model.predict(np.expand_dims(input_image, axis=0))[0]
    index = match(feature, database, threshold)
    if index is not None:
        print(f"Matched with image {index}")
    else:
        print("No match found")


Found 0 files belonging to 0 classes.


ValueError: No images found in directory Images/Humans. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

### Step 6: Train a K-Nearest Neighbors (KNN) classifier

In [ ]:
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)


### Step 7: Evaluate the model on the testing set

In [ ]:
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
